## Ini File Formatter

The micromet .ini files are **not** in standard .ini format.  An ini file needs key,value pairs grouped by **[Section]** headers.  Headers cannot be repeated.  The **[Trace]..[End]** format used is not standard.


% Matlab comments may cause boundary cases, but I thin I've found most.

Some second stage files have ''omitnan'' where really they should have "omitnan", I've hard coded a replacement, but it will fail if that syntax is present elsewhere.  It'll be flagged in the ini_compare.m test, unless one goes back and replaces it in the original ini file

In [2]:
import traceback
import itertools
import os
import configparser


class format_ini():
    def __init__(self):
        self.config = configparser.ConfigParser()
        self.config.optionxform = str

    def ini_to_config(self,ini_file):
        
        with open(ini_file,'r',encoding='utf8') as f:
            ini = ''
            for line in f.readlines():
                l = line.rstrip().lstrip()
                if l.startswith('%')==False:
                    
                    l = l.replace("{'",'{"').replace("'}",'"}').replace(";'",';"').replace("''omitnan''",'"omitnan"')
                    L = l.split("'")
                    # Remove any inline trailing comments
                    if L[-1].endswith(';"'):
                        ap = ';"' # If comment is closed by a semicolon in an eval, keep the line ending
                    else:
                        ap = ''
                    L[-1]=L[-1].split('%')[0]+ap
                    l = ("").join(L)
                    l = l.replace('%','%%').replace('{"',"{'").replace('"}',"'}").replace('; ;',';').replace(';";',';')
                    l = l.rstrip()
                    if l != '':
                        if l.endswith(',') or l.endswith(';') or l.endswith('...') or l.endswith('end'):
                            if l.endswith(',') == False:
                                l = l+','
                            ini = ini+l
                        else:
                            ini = ini+l.replace(';"',';')+'\n'

            
        header = ini.split('[Trace]',1)[0]
        header = ('\n').join([h for h in header.split('\n') if h.lstrip().startswith('%')==False])
        items = [rec for rec in header.split('\n') if '=' in rec]
        default = '[Metadata]\n'
        for row in items:
            default += row.replace("'",'') + '\n'
        self.config.read_string(default)
        blocks = [self.get_sec(b.split('[End]')[0]) for b in ini.split('[Trace]') if '[End]' in b]
        for blk in blocks:
            try:
                sec = blk.split(']')[0].split('[')[-1]
                if sec in self.config.sections():
                    print(f'{sec} is duplicated in input file!!!\n Check:\n')
                    print(blk)
                self.config.read_string(blk)
            except:
                print('Could Not Parse Block:')
                print(blk)

                print('See traceback:\n')
                print(traceback.format_exc())
                print('\n\n')
                pass
        
    def get_sec(self,b_in):
        sec = '['+b_in.split('variableName')[1].split('\n')[0].lstrip('= ')+']\n'+b_in
        return(sec)
    
    
SiteID = 'BBS'
Stage = 'Second'
ini_file_old = f'{SiteID}/{SiteID}_{Stage}Stage_oldFormat.ini'
ini_file = f'{SiteID}/{SiteID}_{Stage}Stage.ini'

if os.path.isfile(ini_file_old)==False and os.path.isfile(ini_file)==True:
    os.rename(ini_file,ini_file_old)

ini=format_ini()
ini.ini_to_config(ini_file_old)

with open(ini_file, 'w',encoding='utf8') as configfile:
    ini.config.write(configfile,space_around_delimiters=False)


In [59]:

# Specify variables only relevant variables for input into REddyProc 
col_order = ["year","DOY","HHMM","NEE","FC","LE","H","FCH4","SW_IN_1_1_1","TA_1_1_1","RH_1_1_1","VPD_1_1_1","USTAR"]

# Specify variable names in REddyProc
var_names = ['Year','DoY','Hour','NEE','FC','LE','H','FCH4','Rg','Tair','rH','VPD','Ustar']

print('[REddyProc_In]')
for r,o in zip(var_names,col_order):
    print(f'{r}={o}')

    
# Define variables to save in third stage
vars_third_stage_REddyProc = ['GPP_uStar_f','GPP_DT_uStar','NEE_uStar_orig','NEE_uStar_f','FC_uStar_orig','FC_uStar_f','LE_uStar_orig','LE_uStar_f','H_uStar_orig','H_uStar_f','FCH4_uStar_orig','FCH4_uStar_f','Reco_uStar','Reco_DT_uStar']
vars_names_third_stage = ['GPP_PI_F_NT','GPP_PI_F_DT','NEE','NEE_PI_F_MDS','FC','FC_PI_F_MDS','LE','LE_PI_F_MDS','H','H_PI_F_MDS','FCH4','FCH4_PI_F_MDS','Reco_PI_F_NT','Reco_PI_F_DT']

print('\n[REddyProc_Out]')

for r,o in zip(vars_third_stage_REddyProc,vars_names_third_stage):
    print(f'{r}={o}')

predictors_FCH4 = ["FCH4", "USTAR","NEE_PI_F_MDS","LE_PI_F_MDS","H_PI_F_MDS","SW_IN_1_1_1","TA_1_1_1","TS_1",
                "RH_1_1_1","VPD_1_1_1","PA_1_1_1"]


print('\n[GapFill_FCH4_Predictors]')
for r in predictors_FCH4:
    print(f'{r}=1')

FCH4 = ["FCH4", "USTAR","NEE_PI_F_MDS","LE_PI_F_MDS","H_PI_F_MDS","SW_IN_1_1_1","TA_1_1_1","TS_1",
                "RH_1_1_1","VPD_1_1_1","PA_1_1_1"]


copy_vars = ['clean_tv','TA_1_1_1','TA_1_2_1','RH_1_1_1','RH_1_2_1','VPD_1_1_1','PA_1_1_1','P_1_1_1','WS_1_1_1','WD_1_1_1',
               'wind_speed','wind_dir','WTD_1_1_1','ORP_1_1_1',
               'pH_1_1_1','TW_1_1_1','CO2','H2O','CH4','SW_IN_1_1_1','SW_OUT_1_1_1','LW_IN_1_1_1','LW_OUT_1_1_1',
               'NETRAD_1_1_1','ALB_1_1_1','PPFD_IN_1_1_1','PPFD_OUT_1_1_1',
               'TS_1','TS_2','TS_3','G_1','USTAR','SLE','SH','SCH4','SC',
               'TKE','L','U_SIGMA','V_SIGMA','W_SIGMA','TAU','zdL']

print('\n[CopyVars]')
for r in copy_vars:
    print(f'{r}=1')

[REddyProc_In]
Year=year
DoY=DOY
Hour=HHMM
NEE=NEE
FC=FC
LE=LE
H=H
FCH4=FCH4
Rg=SW_IN_1_1_1
Tair=TA_1_1_1
rH=RH_1_1_1
VPD=VPD_1_1_1
Ustar=USTAR

[REddyProc_Out]
GPP_uStar_f=GPP_PI_F_NT
GPP_DT_uStar=GPP_PI_F_DT
NEE_uStar_orig=NEE
NEE_uStar_f=NEE_PI_F_MDS
FC_uStar_orig=FC
FC_uStar_f=FC_PI_F_MDS
LE_uStar_orig=LE
LE_uStar_f=LE_PI_F_MDS
H_uStar_orig=H
H_uStar_f=H_PI_F_MDS
FCH4_uStar_orig=FCH4
FCH4_uStar_f=FCH4_PI_F_MDS
Reco_uStar=Reco_PI_F_NT
Reco_DT_uStar=Reco_PI_F_DT

[GapFill_FCH4_Predictors]
FCH4=1
USTAR=1
NEE_PI_F_MDS=1
LE_PI_F_MDS=1
H_PI_F_MDS=1
SW_IN_1_1_1=1
TA_1_1_1=1
TS_1=1
RH_1_1_1=1
VPD_1_1_1=1
PA_1_1_1=1

[CopyVars]
clean_tv=1
TA_1_1_1=1
TA_1_2_1=1
RH_1_1_1=1
RH_1_2_1=1
VPD_1_1_1=1
PA_1_1_1=1
P_1_1_1=1
WS_1_1_1=1
WD_1_1_1=1
wind_speed=1
wind_dir=1
WTD_1_1_1=1
ORP_1_1_1=1
pH_1_1_1=1
TW_1_1_1=1
CO2=1
H2O=1
CH4=1
SW_IN_1_1_1=1
SW_OUT_1_1_1=1
LW_IN_1_1_1=1
LW_OUT_1_1_1=1
NETRAD_1_1_1=1
ALB_1_1_1=1
PPFD_IN_1_1_1=1
PPFD_OUT_1_1_1=1
TS_1=1
TS_2=1
TS_3=1
G_1=1
USTAR=1
SLE=1
SH=1
SCH4=1
